In [1]:
import gradio as gr
from transformers import pipeline
import ffmpeg

Gradio is an open-source Python library that is used to build machine learning and data science demos and web applications.

In [2]:
### Text generation
gr.close_all()

#Firstly, define a function
def predict(prompt):
    model = pipeline("text-generation")
    completion = model(prompt)[0]["generated_text"]
    return completion

#Secondly, create an 'interface'
textbox = gr.Textbox(label="Type your sentence here:", placeholder="Hi, my name is John Doe.", lines=2)
demo = gr.Interface(fn=predict, inputs=textbox, outputs="text")

#Then launch
demo.launch(server_port=7860)

Running on local URL:  http://127.0.0.1:7860/

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x7fad38913fd0>, 'http://127.0.0.1:7860/', None)

In [3]:
### Audio transcription
#demo.close()
#gr.close_all()

#Firstly, define a funtion
def transcribe_audio(mic=None, file=None):
    model = pipeline("automatic-speech-recognition")
    if mic is not None:
        audio = mic
    elif file is not None:
        audio = file
    else:
        return "You must either provide a mic recording or a file"
    transcription = model(audio)["text"]
    return transcription

#Secondly, create an 'interface'
demo2 = gr.Interface(
    fn=transcribe_audio,
    inputs=[
        gr.Audio(source="microphone", type="filepath"),
        gr.Audio(source="upload", type="filepath"),
    ],
    outputs="text",
)

#Then launch
demo2.launch(server_port=7862)

Running on local URL:  http://127.0.0.1:7862/

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x7fad38566a90>, 'http://127.0.0.1:7862/', None)

In [4]:
### Prediction of bioassay with Random Forest example
# Input: Cell Painting features extracted by CellProfiler
# Output: Activity ("Active", or "Inactive") of ChEMBL assay CHEMBL1741322
# Assay description: Cytochrome panel assay. Compounds with AC50 equal or less than 10 uM are considered active

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

dataset = pd.read_csv("toy_dataset_Cytochrome.csv")
X_train, X_test, y_train, y_test = train_test_split(dataset.iloc[:,0:-1], dataset["CHEMBL1741322"], test_size=0.1)

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train.values, y_train.values)

np.savez("CP_predict.npz", X=X_test.iloc[[1,11,67,109,222],:].values, y=y_test.iloc[[1,11,67,109,222]].values)

In [5]:
#demo2.close()
#gr.close_all()

def predict(cp_profile):
    features = np.load(cp_profile)["X"]
    true_label = np.load(cp_profile)["y"]
    pred = clf.predict(features)
    pred = ["Active" if i==1 else "Inactive" for i in pred]
    return pred#, true_label
    
textbox = gr.Textbox(label="Path to Cell Painting features here:", lines=2, placeholder="CP_predict.npz")
demo3 = gr.Interface(fn=predict, inputs=textbox, outputs="text")

demo3.launch(server_port=7864)

Running on local URL:  http://127.0.0.1:7864/

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x7fac87e27d90>, 'http://127.0.0.1:7864/', None)

No model was supplied, defaulted to gpt2 (https://huggingface.co/gpt2)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
